In [89]:
import torch
device='cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [90]:
with open('wizard_of_oz.txt','r', encoding='utf-8') as f:
    text=f.read()



In [91]:
chars=sorted(set(text))

In [92]:
#tokenizers

char_to_int={ch:i for i,ch in enumerate(chars)}
int_to_char={i:ch for i,ch in enumerate(chars)}

encode= lambda s:[char_to_int[c] for c in s]
decode= lambda i:[int_to_char[c] for c in i]

In [93]:
enc=encode('Hello')

dec=decode(enc)

print(enc,dec)

[32, 58, 65, 65, 68] ['H', 'e', 'l', 'l', 'o']


In [94]:
data=torch.tensor(encode(text), dtype=torch.long)
data.shape[0]

232309

In [95]:
#train and test split
len=int(0.8*data.shape[0])
print(len)

train_data=data[:len]
test_data=data[len:]

185847


In [96]:
blocksize=8
batchsize=4

x=train_data[:blocksize]
y=train_data[1:blocksize+1]

for t in range(blocksize):
    content=x[:t+1]
    target=y[t]

    print(f"When Conten is {content} targert is {target}")


When Conten is tensor([80]) targert is 1
When Conten is tensor([80,  1]) targert is 1
When Conten is tensor([80,  1,  1]) targert is 28
When Conten is tensor([80,  1,  1, 28]) targert is 39
When Conten is tensor([80,  1,  1, 28, 39]) targert is 42
When Conten is tensor([80,  1,  1, 28, 39, 42]) targert is 39
When Conten is tensor([80,  1,  1, 28, 39, 42, 39]) targert is 44
When Conten is tensor([80,  1,  1, 28, 39, 42, 39, 44]) targert is 32
